<a href="https://colab.research.google.com/github/sarahbraia/dissertation_methodology_analysis_ITA/blob/main/dissertation_methodology_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

APPLICATION OF THE STUDY OF TIMES AND MOVEMENTS AND WRIGHT’S LAW

In [101]:
#Importando os dados do google drive
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from IPython.display import display

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
# Reading files from Google Drive
base_path = '/content/drive/MyDrive/ITA -DISSERTAÇÃO/Dados_cronoanalise_cablagem.xlsx'
df = pd.read_excel(base_path)

# Preview of each dataset
df.head()

,OP,ETAPA,ID_OPERACAO,SETUP_HORA(S.H),EXECUCAO_HORA(E.H),TEMPO_TOTAL(T.T),OPERADOR
0,74143086,Abrir chamado,Abrir chamado,00:09:00,00:04:29,00:13:29,Operador 1
1,74143082,Aparafusar abraçadeira do conector,Aparafusar,00:00:40,00:05:34,00:06:14,Operador 1
2,74143082,Aparafusar abraçadeira do conector,Aparafusar,00:01:03,00:03:00,00:04:03,Operador 1
3,74138721,Aparafusar capa do conector metalizado,Aparafusar,00:01:24,00:02:20,00:03:44,Operador 5
4,74138721,Aparafusar capa do conector metalizado,Aparafusar,00:01:28,00:02:08,00:03:36,Operador 5


### Application of Micromovement Study in Cable Assembly

In [103]:
# Function to convert time to seconds
def time_to_seconds(t):
    return t.hour * 3600 + t.minute * 60 + t.second

# Apply the function to the time columns
df['SETUP_SECONDS'] = df['SETUP_HORA(S.H)'].apply(time_to_seconds)
df['EXECUTION_SECONDS'] = df['EXECUCAO_HORA(E.H)'].apply(time_to_seconds)
df['TOTAL_SECONDS'] = df['TEMPO_TOTAL(T.T)'].apply(time_to_seconds)


# Display the result
df[['ID_OPERACAO', 'SETUP_SECONDS', 'EXECUTION_SECONDS', 'TOTAL_SECONDS']].head()

,ID_OPERACAO,SETUP_SECONDS,EXECUTION_SECONDS,TOTAL_SECONDS
0,Abrir chamado,540,269,809
1,Aparafusar,40,334,374
2,Aparafusar,63,180,243
3,Aparafusar,84,140,224
4,Aparafusar,88,128,216


In [104]:
# Dictionary for replacing and standardizing operation names
operation_name_corrections = {
    'Abrir chamado ': 'Abrir chamado',
    'Marcar o fio ': 'Marcar fio',
    'Medir tamanho do cabo ': 'Medir tamanho do cabo',
    'Montar conector': 'Montar Conector',
    'Organizar a ordem dos fios': 'Organizar os fios',
    'Soprador Térmico': 'Soprador térmico',
    'Limpeza': 'Limpar excesso de solda'
}

# Replace incorrect or inconsistent operation names in the 'ID_OPERACAO' column
df['ID_OPERACAO'] = df['ID_OPERACAO'].replace(operation_name_corrections)

In [105]:
# Activity to ID mapping dictionary
mapa_ids = {
    'Abrir chamado': 1,
    'Preparar bancada': 2,
    'Medir tamanho do cabo': 3,
    'Marcar fio': 4,
    'Cortar': 5,
    'Decapar': 6,
    'Estanhar': 7,
    'Crimpar': 8,
    'Inserir': 9,
    'Organizar os fios': 10,
    'Soldar': 11,
    'Soprador térmico': 12,
    'Montar Conector': 13,
    'Limpar excesso de solda': 14,
    'Teste de continuidade': 15,
    'Aparafusar': 16,
    'Torque no conector': 17,
    'Fabricar adesivo': 18,
    'Enrolar cabo': 19
}

# Create new column 'ID' as integer
df['ID'] = df['ID_OPERACAO'].map(mapa_ids).astype('Int64')

# Remove null records in 'ID_OPERACAO'
df = df[df['ID_OPERACAO'].notna()]

df.head()


,OP,ETAPA,ID_OPERACAO,SETUP_HORA(S.H),EXECUCAO_HORA(E.H),TEMPO_TOTAL(T.T),OPERADOR,SETUP_SECONDS,EXECUTION_SECONDS,TOTAL_SECONDS,ID
0,74143086,Abrir chamado,Abrir chamado,00:09:00,00:04:29,00:13:29,Operador 1,540,269,809,1
1,74143082,Aparafusar abraçadeira do conector,Aparafusar,00:00:40,00:05:34,00:06:14,Operador 1,40,334,374,16
2,74143082,Aparafusar abraçadeira do conector,Aparafusar,00:01:03,00:03:00,00:04:03,Operador 1,63,180,243,16
3,74138721,Aparafusar capa do conector metalizado,Aparafusar,00:01:24,00:02:20,00:03:44,Operador 5,84,140,224,16
4,74138721,Aparafusar capa do conector metalizado,Aparafusar,00:01:28,00:02:08,00:03:36,Operador 5,88,128,216,16


In [106]:
# Replace values in-place using the translation dictionary
df['ID_OPERACAO'] = df['ID_OPERACAO'].replace({
    "Abrir chamado": "Open request",
    "Preparar bancada": "Prepare workstation",
    "Medir tamanho do cabo": "Measure cable length",
    "Marcar fio": "Mark wire",
    "Cortar": "Cut",
    "Decapar": "Strip",
    "Estanhar": "Tin",
    "Crimpar": "Crimp",
    "Inserir": "Insert",
    "Organizar os fios": "Organize wires",
    "Soldar": "Solder",
    "Soprador térmico": "Heat gun",
    "Montar Conector": "Assemble connector",
    "Limpar excesso de solda": "Clean solder excess",
    "Teste de continuidade": "Continuity test",
    "Aparafusar": "Screw",
    "Torque no conector": "Connector torque",
    "Fabricar adesivo": "Create label",
    "Enrolar cabo": "Coil cable"
})
df.head()

,OP,ETAPA,ID_OPERACAO,SETUP_HORA(S.H),EXECUCAO_HORA(E.H),TEMPO_TOTAL(T.T),OPERADOR,SETUP_SECONDS,EXECUTION_SECONDS,TOTAL_SECONDS,ID
0,74143086,Abrir chamado,Open request,00:09:00,00:04:29,00:13:29,Operador 1,540,269,809,1
1,74143082,Aparafusar abraçadeira do conector,Screw,00:00:40,00:05:34,00:06:14,Operador 1,40,334,374,16
2,74143082,Aparafusar abraçadeira do conector,Screw,00:01:03,00:03:00,00:04:03,Operador 1,63,180,243,16
3,74138721,Aparafusar capa do conector metalizado,Screw,00:01:24,00:02:20,00:03:44,Operador 5,84,140,224,16
4,74138721,Aparafusar capa do conector metalizado,Screw,00:01:28,00:02:08,00:03:36,Operador 5,88,128,216,16


#### Refined cabling operation

In [107]:
# Group by ID and ID_OPERACAO and count occurrences
distinct_counts = (
    df.groupby(['ID', 'ID_OPERACAO'])
    .size()
    .reset_index(name='COUNT')
    .sort_values(by='ID')
)

# Display the result
distinct_counts


,ID,ID_OPERACAO,COUNT
0,1,Open request,1
1,2,Prepare workstation,12
2,3,Measure cable length,21
3,4,Mark wire,57
4,5,Cut,47
5,6,Strip,77
6,7,Tin,41
7,8,Crimp,21
8,9,Insert,69
9,10,Organize wires,7


#### Number of cycles to be timed

##### Calculate the range (R) per refined operation, the mean (X) and ratio (R/X)

In [108]:
# Group by operation and calculate the highest (H) and lowest (L) times
analysis_df = df.groupby('ID_OPERACAO').agg(
    H=('TOTAL_SECONDS', 'max'),
    L=('TOTAL_SECONDS', 'min')
).reset_index()

# Calculate range (R = H - L)
analysis_df['R'] = analysis_df['H'] - analysis_df['L']

# Calculate approximate mean (X̄ ≈ (H + L) / 2)
analysis_df['X̄'] = (analysis_df['H'] + analysis_df['L']) / 2

# Calculate relative variability ratio (R / X̄)
analysis_df['R/X̄'] = (analysis_df['R'] / analysis_df['X̄']).round(2)

# Display final result
display(analysis_df)


,ID_OPERACAO,H,L,R,X̄,R/X̄
0,Assemble connector,66,66,0,66.0,0.00
1,Clean solder excess,105,9,96,57.0,1.68
2,Coil cable,29,29,0,29.0,0.00
3,Connector torque,216,203,13,209.5,0.06
4,Continuity test,640,35,605,337.5,1.79
5,Create label,237,15,222,126.0,1.76
6,Crimp,149,10,139,79.5,1.75
7,Cut,208,7,201,107.5,1.87
8,Heat gun,267,10,257,138.5,1.86
9,Insert,59160,5,59155,29582.5,2.00


In [115]:
# Reading files from Google Drive
table_relative_error = '/content/drive/MyDrive/ITA -DISSERTAÇÃO/table_relative_error.xlsx'
table_relative_error_df = pd.read_excel(table_relative_error)
# Preview of each dataset
table_relative_error_df.head()

,R/X,relative_error_of_5%,relative_error_of_10%
0,0.10,3,2
1,0.12,4,2
2,0.14,6,3
3,0.16,8,4
4,0.18,10,6
